In [1]:
!pip install -q transformers datasets tensorflow


In [2]:
pip install tensorflow-macos tensorflow-metal


ERROR: Could not find a version that satisfies the requirement tensorflow-macos
ERROR: No matching distribution found for tensorflow-macos
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install -U pip setuptools wheel
!pip install -U transformers datasets tensorflow tqdm nltk


     |████████████████████████████████| 1.8 MB 1.7 MB/s 
     |████████████████████████████████| 1.3 MB 28.3 MB/s 
     |████████████████████████████████| 72 kB 6.8 MB/s 
  Attempting uninstall: wheel
    Found existing installation: wheel 0.36.2
    Uninstalling wheel-0.36.2:
      Successfully uninstalled wheel-0.36.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 52.0.0.post20210125
    Uninstalling setuptools-52.0.0.post20210125:
      Successfully uninstalled setuptools-52.0.0.post20210125
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requi

In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import tensorflow as tf
from transformers import (
    DistilBertTokenizerFast,
    TFDistilBertForSequenceClassification,
    DataCollatorWithPadding
)

In [3]:
ds = load_dataset("lucadiliello/newsqa")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/681 [00:00<?, ?B/s]

data/train-00000-of-00001-ec54fbe500fc3b(…):   0%|          | 0.00/29.7M [00:00<?, ?B/s]

data/validation-00000-of-00001-3cf888b12(…):   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/74160 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4212 [00:00<?, ? examples/s]

In [4]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 74160
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 4212
    })
})


In [5]:
train_df = ds["train"].to_pandas().sample(3000, random_state=42)
valid_df = ds["validation"].to_pandas().sample(500, random_state=42)

In [6]:
train_df.head()

,context,question,answers,key,labels
48592,"BEIJING, China (CNN) -- The world will soon se...",How many people died from the virus since last...,"[More than 1,490]",2ddf1c16124f46cc8c86678d4838971e,"[{'end': [851], 'start': [837]}]"
22186,Washington (CNN) -- The word compromise is nev...,How many troops were fired?,"[More than 14,000]",cba1b689f43a4b1892165b87ee98bc21,"[{'end': [625], 'start': [610]}]"
33832,"(CNN) -- Hemant Karkare, Mumbai's slain terror...",Where was Karkare going when he heard of the a...,[heading home],99bc18956daa4fe5b3126cf589b18fcc,"[{'end': [718], 'start': [707]}]"
54572,(Financial Times) -- When Macau unveiled recor...,How many mainland visitors did Macau have last...,[16m],009585a7d99b4a5d962a5685feaa2bf0,"[{'end': [1962], 'start': [1960]}]"
46261,(CNN) -- The driver of a slain Catholic bishop...,Who was killed in Anatolia?,"[Luigi Padovese,]",06de47ca71834490ad1ec1fb296461aa,"[{'end': [210], 'start': [196]}]"


In [7]:
valid_df.head()

,context,question,answers,key,labels
643,"BERLIN, Germany (CNN) -- A German cruise liner...",How many passengers does it involve?,[246],f2a8589ed0aa4c0e9cfb1446827b09ab,"[{'end': [305], 'start': [303]}]"
166,WASHINGTON (CNN) -- President Bush on Monday s...,What did he sign?,[a bill that will pay for the wars in Iraq and...,f9ef08e2693943069670c3b422f8513a,"[{'end': [108], 'start': [52]}]"
1444,"San Diego, California (CNN) -- Don't be surpri...",What causes people to hire illegals?,[to do jobs that Arizonans wouldn't do.],f2d500e47e4d4a18bd068db4ff12063c,"[{'end': [1779], 'start': [1742]}]"
1877,"LONDON, England (CNN) -- Bollywood leading man...",what is the second largest film industry,[Bollywood],b9baaeedf23f47088fe44a56c08433ca,"[{'end': [1210], 'start': [1202]}]"
463,"ORLANDO, Florida (CNN) -- The meter reader who...",What child's remains were found?,[Caylee Anthony],f2b9e8d1f4fe486d87a9d663503bef07,"[{'end': [121], 'start': [108]}]"


In [8]:
def has_answer(x):
    if isinstance(x, dict) and "text" in x and isinstance(x["text"], list):
        return 1 if len(x["text"]) > 0 else 0
    else:
        return 0

train_df["answerable"] = train_df["answers"].apply(has_answer)
valid_df["answerable"] = valid_df["answers"].apply(has_answer)


In [9]:
y_train = train_df["answerable"].values
y_val = valid_df["answerable"].values

In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [21]:
train_encodings = tokenizer(
    train_df["context"].tolist(),
    train_df["question"].tolist(),
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors="tf"
)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [35]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"]
    },
    tf.convert_to_tensor(y_train, dtype=tf.int32)
))

In [36]:
val_encodings = tokenizer(
    valid_df["question"].tolist(),
    valid_df["context"].tolist(),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="tf"
)


val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": val_encodings["input_ids"],
        "attention_mask": val_encodings["attention_mask"]
    },
    tf.convert_to_tensor(y_val, dtype=tf.int32)
))

In [37]:
train_dataset = train_dataset.shuffle(1000).batch(8)
val_dataset = val_dataset.batch(8)


In [38]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, from_pt = True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

In [39]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [40]:
train_dataset = train_dataset.shuffle(1000).batch(8)
val_dataset = val_dataset.batch(8)

In [41]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [42]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2,          # 2 epochs to avoid GPU timeout
    verbose=1
)

Epoch 1/2


ValueError: in user code:

    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.12/dist-packages/transformers/modeling_tf_utils.py", line 1656, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file1iscgwer.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "/tmp/__autograph_generated